# $U(1)$ Gauge Model using L2HMC in graph mode

In [1]:
import os
import sys
import time
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from scipy.special import i0, i1

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils.data_utils import (
    calc_avg_vals_errors, block_resampling, jackknife_err
)

%autoreload 2
%matplotlib notebook

from l2hmc_eager import gauge_dynamics_eager as gde

Using TensorFlow backend.


In [2]:
#tf.enable_eager_execution()
tfe = tf.contrib.eager

from u1_model_eager import GaugeModelEager, train_one_iter
from gauge_model import GaugeModel

import utils.gauge_model_helpers as helpers

In [3]:
def autocorr(x):
    result = np.correlate(x, x, mode='full')
    result /= result[result.argmax()]
    return result[result.size//2:]

## Define parameters

In [47]:
params = {
    'time_size': 8,
    'space_size': 8,
    'link_type': 'U1',
    'dim': 2,
    'beta': 8.,
    'num_samples': 10,
    'num_steps': 5,
    'eps': 0.2,
    'loss_scale': 0.1,
    'loss_eps': 1e-4,
    'learning_rate_init': 5e-5,
    'learning_rate_decay_steps': 500,
    'learning_rate_decay_rate': 0.98,
    'train_steps': 5000,
    #'record_loss_every': 50,
    #'data_steps': 100,
    'save_steps': 1000,
    #'print_steps': 5,
    'logging_steps': 50,
    'clip_value': 10,
    'rand': False,
    'metric': 'l2',
}
tf.reset_default_graph()

## Create model

In [48]:
#del model

In [49]:
config = tf.ConfigProto()
tf.reset_default_graph()

In [50]:
model = GaugeModel(params=params,
                   config=config,
                   sess=None,
                   conv_net=False,
                   hmc=False,
                   log_dir=None,
                   restore=False,
                   eps_trainable=True,
                   aux=False)
# 225.39 --> 419.52 w/ aux. fn
# 137.54 --> 228.63 w/ aux. fn

Creating directory for new run: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_48/
log_dir: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_48/
info_dir: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_48/run_info/
figs_dir: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_48/figures/
conv_net: False
hmc: False
aux: False
eps_trainable: True
time_size: 8
space_size: 8
link_type: U1
dim: 2
beta: 8.0
num_samples: 10
num_steps: 5
eps: 0.2
loss_scale: 0.1
loss_eps: 0.0001
learning_rate_init: 5e-05
learning_rate_decay_steps: 500
learning_rate_decay_rate: 0.98
train_steps: 5000
save_steps: 1000
logging_steps: 50
clip_value: 10
rand: False
metric: l2
samples_history_dir: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_48/samples_history
--------------------------------------------------------------------------------

Building graph... (started at: Sat Jan 12 12:33:29 2019)
  Creating loss...

    took: 1087.8854410648346 seconds.
  Creating optimizer...

    took: 462.16167879104614 seconds.
  Cre

In [51]:
model.train(model.train_steps, kill_sess=False)

--------------------------------------------------------------------------------------
     STEP           LOSS       NORM. TIME     ACCEPT %        EPS            LR      
--------------------------------------------------------------------------------------
Time to complete logging: 247.1 

     9/5000       -630.1       0.0001324      0.03227       0.1996         5e-05    
    19/5000        149.3       0.0001224      0.01847       0.1993         5e-05    
    29/5000        997.1       0.0001241     2.523e-10       0.199         5e-05    
    39/5000        1000        0.000119      1.404e-12      0.1988         5e-05    
    49/5000        898.8       0.0001697     3.145e-06      0.1987         5e-05    
Time to complete logging: 0.3986

    59/5000        522.8       0.0001193      0.05182       0.1987         5e-05    
    69/5000        1000        0.0001323     6.693e-13      0.1986         5e-05    
    79/5000        1000        0.0001281     8.653e-17      0.1985         5e

   879/5000       -689.2       0.0001107      0.1676         0.173        4.9e-05   
   889/5000        -1672       0.0001123      0.3897        0.1727        4.9e-05   
   899/5000        -2408       0.0001125      0.4785        0.1725        4.9e-05   
Time to complete logging: 0.2652

   909/5000        -1948       0.0001121      0.4681        0.1723        4.9e-05   
   919/5000        -1780       0.0001234      0.3743         0.172        4.9e-05   
   929/5000       -764.4       0.0001129      0.1193        0.1716        4.9e-05   
   939/5000        -1240       0.0001163      0.2463        0.1713        4.9e-05   
   949/5000        -1251       0.000115       0.2921        0.1711        4.9e-05   
Time to complete logging: 0.3189

   959/5000        -1198       0.0001152      0.2367        0.1711        4.9e-05   
   969/5000        -3167       0.0001153      0.5868        0.1711        4.9e-05   
   979/5000        -2361       0.0001183      0.4604         0.171        4.9e-05 

  1739/5000        -2251       0.0001126      0.5831        0.1479       4.706e-05  
  1749/5000        -2702       0.0001173      0.6285        0.1477       4.706e-05  
Time to complete logging: 0.2968

  1759/5000        -3197        0.00011       0.6349        0.1475       4.706e-05  
  1769/5000        -2719       0.0001095      0.6282        0.1473       4.706e-05  
  1779/5000        -1633       0.0001062      0.4459         0.147       4.706e-05  
  1789/5000        -4714       0.0001131      0.9561        0.1466       4.706e-05  
  1799/5000        -3187       0.0001113      0.6321        0.1464       4.706e-05  
Time to complete logging: 0.277 

  1809/5000        -2918       0.0001211      0.6145        0.1462       4.706e-05  
  1819/5000        -3229       0.0001147      0.7524         0.146       4.706e-05  
  1829/5000        -2052       0.000113       0.4935        0.1458       4.706e-05  
  1839/5000        -2544       0.0001141      0.6835        0.1455       4.706e-05

  2599/5000        -3897       0.0001129      0.9936         0.128       4.52e-05   
Time to complete logging: 0.2883

  2609/5000        -3205       0.0001174      0.8505        0.1278       4.52e-05   
  2619/5000        -3226       0.0001096       0.895        0.1275       4.52e-05   
  2629/5000        -2678       0.0001094      0.6771        0.1271       4.52e-05   
  2639/5000        -2484       0.0001185      0.7383        0.1269       4.52e-05   
  2649/5000        -3113       0.0001138      0.8183        0.1266       4.52e-05   
Time to complete logging: 0.3311

  2659/5000        -3808       0.0001053      0.8992        0.1263       4.52e-05   
  2669/5000        -2946       0.0001157      0.7354        0.1259       4.52e-05   
  2679/5000        -2973       0.0001146      0.7525        0.1257       4.52e-05   
  2689/5000        -3513       0.0001198      0.8864        0.1254       4.52e-05   
  2699/5000        -2687       0.0001159      0.7614         0.125       4.52e-05 

Time to complete logging: 0.2573

  3459/5000        -2784       0.0001028      0.8001        0.09936      4.429e-05  
  3469/5000        -3012       0.0001036      0.9639        0.09909      4.429e-05  
  3479/5000        -2943       0.000107       0.9001        0.09873      4.429e-05  
  3489/5000        -3036       0.0001074      0.9821        0.09847      4.429e-05  
  3499/5000        -2528       0.0001074      0.8529        0.09816      4.429e-05  
Time to complete logging: 0.2631

  3509/5000        -2690       9.973e-05       0.889        0.09785      4.341e-05  
  3519/5000        -3321       0.000105       0.9704        0.09747      4.341e-05  
  3529/5000        -2708       0.0001073      0.9371        0.09709      4.341e-05  
  3539/5000        -2858       0.0001064      0.9369        0.09686      4.341e-05  
  3549/5000        -2906       0.0001076      0.9449        0.09664      4.341e-05  
Time to complete logging: 0.2308

  3559/5000        -2906       9.977e-05      0.

  4309/5000        -2456       0.000102       0.9293        0.08176      4.254e-05  
  4319/5000        -1921        0.00011       0.8986        0.08163      4.254e-05  
  4329/5000        -2159       0.0001075       0.907        0.0815       4.254e-05  
  4339/5000        -2908       0.0001316      0.9153        0.08142      4.254e-05  
  4349/5000        -2083       0.0001214      0.9449        0.0813       4.254e-05  
Time to complete logging: 0.2664

  4359/5000        -2321       0.0001567      0.9212        0.08112      4.254e-05  
  4369/5000        -2107       0.0001056      0.9073        0.08082      4.254e-05  
  4379/5000        -1907       0.0001424      0.9742        0.08057      4.254e-05  
  4389/5000        -2349       0.0001203      0.9565        0.08035      4.254e-05  
  4399/5000        -2164       0.0001228      0.9513        0.08032      4.254e-05  
Time to complete logging: 0.2813

  4409/5000        -2226       0.0001118      0.9434        0.08025      4.254e-05

In [52]:
run_steps = [100, 200, 500, 1000, 5000, 10000]

In [ ]:
for steps in run_steps:
    _ = model.run(steps)

Running (trained) L2HMC sampler for 100 steps...


In [ ]:
model.sess.graph.collections

In [ ]:
model.sess.graph.get_collection

In [ ]:
model.dynamics.position_fn.summary()

In [ ]:
# Iterate over samples history and calculate observables for each sample.
# `lattice.calc_plaq_observables(samples)` calculates observables for each of
# the samples in the mini-batch.
actions_history = []
avg_plaquettes_history = []
top_charges_history = []
for idx, samples in enumerate(samples_history):
    t0 = time.time()
    observables = np.array(model.lattice.calc_plaq_observables(samples))
    actions, plaqs, charges = observables
    
    actions_history.append(actions)
    avg_plaquettes_history.append(plaqs)
    top_charges_history.append(charges)
    
    print(f'step: {idx}  '
          f'time / step: {time.time() - t0:^6.4g}  '
          f'avg action: {np.mean(actions):^6.4g}  '
          f'avg plaquette: {np.mean(plaqs):^6.4g} '
          f'top charge: {np.mean(charges):^6.4g}')

In [ ]:
_ = helpers.plot_run_data(model.data, 
                          model.params, 
                          model.steps_arr, 
                          model.figs_dir, 
                          skip_steps=1)

In [ ]:
#tf.reset_default_graph()
#model = GaugeModel(params=params,
#                   config=None,
#                   sess=None,
#                   conv_net=False,
#                   hmc=False,
#                   log_dir='../../gauge_logs_graph/run_25',
#                   restore=True)

In [ ]:
samples = np.random.randn(*model.samples.shape)
samples_history = []

In [ ]:
for i in range(500):
    t0 = time.time()
    samples = model.sess.run(model.x_out, feed_dict={model.x: samples})
    samples_history.append(samples)
    print(f'step: {i:^6.4g} time/step: {time.time() - t0:^6.4g}')

In [ ]:
samples_history_conv = np.array(samples_history_conv)
print(samples_history_conv.shape)

In [ ]:
import pickle
samples_history_file = os.path.join(model.info_dir, 'samples_history.pkl')
with open(samples_history_file, 'wb') as f:
    pickle.dump(samples_history_conv, f)